# मॉडल संदर्भ प्रोटोकॉल (MCP) समर्थन के साथ Azure AI एजेंट

यह नोटबुक दिखाता है कि मॉडल संदर्भ प्रोटोकॉल (MCP) टूल्स के साथ Azure AI एजेंट का उपयोग कैसे करें ताकि एक बुद्धिमान एजेंट बनाया जा सके जो बाहरी MCP सर्वरों का उपयोग करके उन्नत क्षमताओं का लाभ उठा सके।


## आवश्यक NuGet पैकेज इंस्टॉल करें

सबसे पहले, हमें Azure AI Agents Persistent पैकेज इंस्टॉल करना होगा, जो Azure AI Agents के साथ काम करने के लिए मुख्य कार्यक्षमता प्रदान करता है।


## कीलेस प्रमाणीकरण के लाभ

यह नोटबुक **कीलेस प्रमाणीकरण** को प्रदर्शित करता है, जो कई फायदे प्रदान करता है:
- ✅ **कोई API कुंजी प्रबंधन की आवश्यकता नहीं** - Azure पहचान-आधारित प्रमाणीकरण का उपयोग करता है
- ✅ **सुरक्षा में सुधार** - कोड या कॉन्फ़िगरेशन में कोई गुप्त जानकारी संग्रहीत नहीं होती
- ✅ **स्वचालित क्रेडेंशियल रोटेशन** - Azure क्रेडेंशियल जीवनचक्र को संभालता है
- ✅ **भूमिका-आधारित पहुंच** - सूक्ष्म अनुमति के लिए Azure RBAC का उपयोग करता है

`DefaultAzureCredential` स्वचालित रूप से सबसे अच्छा उपलब्ध क्रेडेंशियल स्रोत का उपयोग करेगा:
1. प्रबंधित पहचान (जब Azure में चल रहा हो)
2. Azure CLI क्रेडेंशियल (विकास के दौरान)
3. Visual Studio क्रेडेंशियल
4. पर्यावरण चर (यदि कॉन्फ़िगर किया गया हो)


In [1]:
#r "nuget: Azure.AI.Agents.Persistent, 1.1.0-beta.4"

Installed Packages Azure.AI.Agents.Persistent, 1.1.0-beta.4

Azure सेवाओं के साथ प्रमाणीकरण के लिए DefaultAzureCredential का उपयोग करके Azure Identity पैकेज इंस्टॉल करें।


In [2]:
#r "nuget: Azure.Identity, 1.14.2"

Installed Packages Azure.Identity, 1.14.2

## आवश्यक नामस्थान आयात करें

Azure AI Agents और Azure Identity के लिए आवश्यक नामस्थान आयात करें।


In [3]:
using Azure.AI.Agents.Persistent;
using Azure.Identity;

## Azure AI एजेंट क्लाइंट (कीलेस ऑथेंटिकेशन) को कॉन्फ़िगर करें

कॉन्फ़िगरेशन वेरिएबल्स सेट करें और **कीलेस ऑथेंटिकेशन** का उपयोग करके PersistentAgentsClient बनाएं:
- **projectEndpoint**: Azure AI Foundry प्रोजेक्ट का एंडपॉइंट
- **modelDeploymentName**: डिप्लॉय किए गए AI मॉडल का नाम (GPT-4.1 नैनो)
- **mcpServerUrl**: MCP सर्वर का URL (Microsoft Learn API)
- **mcpServerLabel**: MCP सर्वर की पहचान के लिए एक लेबल
- **DefaultAzureCredential**: मैनेज्ड आइडेंटिटी, Azure CLI, या अन्य क्रेडेंशियल स्रोतों का उपयोग करता है (API कीज़ की आवश्यकता नहीं)


In [ ]:
var projectEndpoint = "Your Azure AI Foundry Project Endpoint";
var modelDeploymentName = "Your Azure OpenAI Model Deployment Name";
var mcpServerUrl = "https://learn.microsoft.com/api/mcp";
var mcpServerLabel = "mslearn";
PersistentAgentsClient agentClient = new(projectEndpoint, new DefaultAzureCredential());

## MCP टूल परिभाषा बनाएं

Microsoft Learn MCP सर्वर से कनेक्ट करने के लिए एक MCP टूल परिभाषा बनाएं। यह एजेंट को Microsoft Learn सामग्री और दस्तावेज़ों तक पहुंचने की अनुमति देगा।


In [ ]:
MCPToolDefinition mcpTool = new(mcpServerLabel, mcpServerUrl);

## AI एजेंट बनाएं

निर्दिष्ट मॉडल और MCP टूल्स के साथ एक स्थायी AI एजेंट बनाएं। एजेंट को निम्नलिखित के साथ कॉन्फ़िगर किया गया है:
- GPT-4.1 नैनो मॉडल
- सहायता के लिए MCP टूल्स का उपयोग करने के निर्देश
- Microsoft Learn MCP सर्वर तक पहुंच


In [ ]:
PersistentAgent agent = await agentClient.Administration.CreateAgentAsync(
   model: modelDeploymentName,
   name: "my-learn-agent",
   instructions: "You are a helpful agent that can use MCP tools to assist users. Use the available MCP tools to answer questions and perform tasks.",
   tools: [mcpTool]
   );

## थ्रेड बनाएं और संदेश भेजें

एक बातचीत थ्रेड बनाएं और उपयोगकर्ता को संदेश भेजें जिसमें Azure OpenAI और OpenAI के बीच के अंतर के बारे में पूछा जाए। यह एजेंट की MCP टूल्स का उपयोग करके सटीक जानकारी प्रदान करने की क्षमता का परीक्षण करेगा।


In [7]:
PersistentAgentThread thread = await agentClient.Threads.CreateThreadAsync();

// Create message to thread
PersistentThreadMessage message = await agentClient.Messages.CreateMessageAsync(
    thread.Id,
    MessageRole.User,
    "What's difference between Azure OpenAI and OpenAI?");

## MCP टूल संसाधनों को कॉन्फ़िगर करें (बिना कुंजी के)

MCP टूल संसाधनों को सेट करें। पूरी तरह से बिना कुंजी वाले दृष्टिकोण के लिए, आप कस्टम हेडर हटा सकते हैं यदि MCP सर्वर Azure पहचान-आधारित प्रमाणीकरण का समर्थन करता है। नीचे दिया गया उदाहरण दिखाता है कि यदि आवश्यक हो तो हेडर कैसे जोड़ें, लेकिन बिना कुंजी वाले परिदृश्यों के लिए, ये आवश्यक नहीं हो सकते हैं।


In [ ]:
// Option 1: Completely keyless (if MCP server supports Azure identity)
MCPToolResource mcpToolResource = new(mcpServerLabel);
ToolResources toolResources = mcpToolResource.ToToolResources();

// Option 2: With custom headers (if still needed for specific MCP servers)
// MCPToolResource mcpToolResource = new(mcpServerLabel);
// mcpToolResource.UpdateHeader("Authorization", "Bearer <your-token>");
// ToolResources toolResources = mcpToolResource.ToToolResources();

## एजेंट रन शुरू करें

उपयोगकर्ता के संदेश को संसाधित करने के लिए एक रन बनाएं और शुरू करें। एजेंट प्रतिक्रिया उत्पन्न करने के लिए कॉन्फ़िगर किए गए MCP टूल्स और संसाधनों का उपयोग करेगा।


In [9]:
ThreadRun run = await agentClient.Runs.CreateRunAsync(thread, agent, toolResources);

## रन की निगरानी करें और टूल अनुमोदन संभालें (Keyless)

एजेंट रन की स्थिति की निगरानी करें और आवश्यक टूल अनुमोदन को संभालें। यह लूप:
1. रन के पूरा होने या कार्रवाई की आवश्यकता होने का इंतजार करता है
2. आवश्यक होने पर MCP टूल कॉल को स्वचालित रूप से अनुमोदित करता है
3. Keyless प्रमाणीकरण के लिए, यदि MCP सर्वर Azure पहचान का समर्थन करता है तो हेडर्स की आवश्यकता नहीं हो सकती


In [ ]:
while (run.Status == RunStatus.Queued || run.Status == RunStatus.InProgress || run.Status == RunStatus.RequiresAction)
{
    await Task.Delay(TimeSpan.FromMilliseconds(1000));
    run = await agentClient.Runs.GetRunAsync(thread.Id, run.Id);

    if (run.Status == RunStatus.RequiresAction && run.RequiredAction is SubmitToolApprovalAction toolApprovalAction)
    {
        var toolApprovals = new List<ToolApproval>();
        foreach (var toolCall in toolApprovalAction.SubmitToolApproval.ToolCalls)
        {
            if (toolCall is RequiredMcpToolCall mcpToolCall)
            {
                Console.WriteLine($"Approving MCP tool call: {mcpToolCall.Name}");
                
                // Option 1: Keyless approval (no headers needed)
                toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true));
                
                // Option 2: With headers (if required by specific MCP server)
                // toolApprovals.Add(new ToolApproval(mcpToolCall.Id, approve: true)
                // {
                //     Headers = { ["Authorization"] = "Bearer <your-token>" }
                // });
            }
        }

        if (toolApprovals.Count > 0)
        {
            run = await agentClient.Runs.SubmitToolOutputsToRunAsync(thread.Id, run.Id, toolApprovals: toolApprovals);
        }
    }
}

Approving MCP tool call: microsoft_docs_search


## बातचीत के परिणाम दिखाएं

थ्रेड में सभी संदेशों को प्राप्त करें और दिखाएं, जिसमें उपयोगकर्ता के प्रश्न और एजेंट के उत्तर दोनों शामिल हों। संदेशों को समयानुसार क्रम में टाइमस्टैम्प और भूमिका संकेतकों के साथ प्रदर्शित किया जाता है।


In [12]:
using Azure;

AsyncPageable<PersistentThreadMessage> messages = agentClient.Messages.GetMessagesAsync(
    threadId: thread.Id,
    order: ListSortOrder.Ascending
);

await foreach (PersistentThreadMessage threadMessage in messages)
{
    Console.Write($"{threadMessage.CreatedAt:yyyy-MM-dd HH:mm:ss} - {threadMessage.Role,10}: ");
    foreach (MessageContent contentItem in threadMessage.ContentItems)
    {
        if (contentItem is MessageTextContent textItem)
        {
            Console.Write(textItem.Text);
        }
        else if (contentItem is MessageImageFileContent imageFileItem)
        {
            Console.Write($"<image from ID: {imageFileItem.FileId}>");
        }
        Console.WriteLine();
    }
}

2025-07-16 06:39:43 -       user: What's difference between Azure OpenAI and OpenAI?
2025-07-16 06:39:51 -  assistant: The main difference between Azure OpenAI and OpenAI lies in their deployment, management, and integration options:

1. **Azure OpenAI**:
   - A cloud service offered through Microsoft Azure.
   - Provides access to OpenAI models with additional enterprise features like security, compliance, and scale.
   - Allows integration with other Azure services, enabling seamless use within existing Azure-based solutions.
   - Offers managed deployment, monitoring, and support within the Azure ecosystem.
   - Suitable for organizations looking for enterprise-grade security, compliance, and regional availability.

2. **OpenAI (OpenAI API)**:
   - A standalone API service provided directly by OpenAI.
   - Accessible via the OpenAI platform without the need for Azure.
   - Focused on providing GPT models, DALL-E, etc., primarily for developers and researchers.
   - Suitable for indi


---

**अस्वीकरण**:  
यह दस्तावेज़ AI अनुवाद सेवा [Co-op Translator](https://github.com/Azure/co-op-translator) का उपयोग करके अनुवादित किया गया है। जबकि हम सटीकता सुनिश्चित करने का प्रयास करते हैं, कृपया ध्यान दें कि स्वचालित अनुवाद में त्रुटियां या अशुद्धियां हो सकती हैं। मूल भाषा में उपलब्ध मूल दस्तावेज़ को प्रामाणिक स्रोत माना जाना चाहिए। महत्वपूर्ण जानकारी के लिए, पेशेवर मानव अनुवाद की सिफारिश की जाती है। इस अनुवाद के उपयोग से उत्पन्न किसी भी गलतफहमी या गलत व्याख्या के लिए हम जिम्मेदार नहीं हैं।
